**Table of contents**<a id='toc0_'></a>    
- 1. [Problem 1: Production economy and CO2 taxation](#toc1_)    
- 2. [Problem 2: Career choice model](#toc2_)    
- 3. [Problem 3: Barycentric interpolation](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [22]:
# Write your code here
import numpy as np
from types import SimpleNamespace

## 1. <a id='toc1_'></a>[Problem 1: Production economy and CO2 taxation](#toc0_)

Consider a production economy with two firms indexed by $j \in \{1,2\}$. Each produce its own good. They solve

$$
\begin{align*}
\max_{y_{j}}\pi_{j}&=p_{j}y_{j}-w_{j}\ell_{j}\\\text{s.t.}\;&y_{j}=A\ell_{j}^{\gamma}.
\end{align*}
$$

Optimal firm behavior is

$$
\begin{align*}
\ell_{j}^{\star}(w,p_{j})&=\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}} \\
y_{j}^{\star}(w,p_{j})&=A\left(\ell_{j}^{\star}(w,p_{j})\right)^{\gamma}
\end{align*}
$$

The implied profits are

$$
\pi_{j}^*(w,p_{j})=\frac{1-\gamma}{\gamma}w\cdot\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}}
$$

A single consumer supplies labor, and consumes the goods the firms produce. She also recieves the implied profits of the firm.<br>
She solves:

$$
\begin{align*}
U(p_1,p_2,w,\tau,T) = \max_{c_{1},c_{2},\ell} & \log(c_{1}^{\alpha}c_{2}^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} \\
\text{s.t.}\,\,\,&p_{1}c_{1}+(p_{2}+\tau)c_{2}=w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})
\end{align*}
$$

where $\tau$ is a tax and $T$ is lump-sum transfer. <br>
For a given $\ell$, it can be shown that optimal behavior is

$$
\begin{align*}
c_{1}(\ell)&=\alpha\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{1}} \\
c_{2}(\ell)&=(1-\alpha)\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{2}+\tau} \\
\end{align*}
$$
Such that optimal behavior is:
$$
\ell^* = \underset{\ell}{\arg\max} \log(\left(c_{1}(\ell)\right)^{\alpha}\cdot \left(c_{2}(\ell)\right)^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} 
$$
With optimal consumption:
$$
\begin{align*}
c_1^*=c_{1}(\ell^*) \\
c_2^*=c_{2}(\ell^*)\\
\end{align*}
$$


The government chooses $\tau$ and balances its budget so $T=\tau c_2^*$. We initially set $\tau,T=0$.

Market clearing requires:

1. Labor market: $\ell^* = \ell_1^* + \ell_2^*$
1. Good market 1: $c_1^* = y_1^*$
1. Good market 2: $c_2^* = y_2^*$


**Question 1:** Check market clearing conditions for $p_1$ in `linspace(0.1,2.0,10)` and $p_2$ in `linspace(0.1,2.0,10)`. We choose $w=1$ as numeraire.

In [23]:
import numpy as np
import pandas as pd
from types import SimpleNamespace

# Define parameters
par = SimpleNamespace()
par.A = 1.0
par.gamma = 0.5
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0
par.tau = 0.0
par.T = 0.0

# Define functions
def optimal_labor(w, p, A, gamma):
    return (p * A**gamma / w)**(1 / (1 - gamma))

def optimal_output(A, l, gamma):
    return A * l**gamma

def firm_profit(w, p, A, gamma):
    return w * (p * A**gamma / w)**(1 - gamma) * (1 - gamma)

def consumer_utility(p1, p2, w, T, pi1, pi2, alpha, nu, epsilon):
    c1 = alpha * (w + T + pi1 + pi2) / p1
    c2 = (1 - alpha) * (w + T + pi1 + pi2) / (p2 + par.tau)
    l = ((c1**alpha * c2**(1 - alpha))**(1 / nu))**(1 / (1 + epsilon))
    return c1, c2, l

# Market clearing conditions
def market_clearing(p1, p2, w, par):
    # Firm 1
    l1 = optimal_labor(w, p1, par.A, par.gamma)
    y1 = optimal_output(par.A, l1, par.gamma)
    pi1 = firm_profit(w, p1, par.A, par.gamma)
    
    # Firm 2
    l2 = optimal_labor(w, p2, par.A, par.gamma)
    y2 = optimal_output(par.A, l2, par.gamma)
    pi2 = firm_profit(w, p2, par.A, par.gamma)
    
    # Consumer
    c1, c2, l = consumer_utility(p1, p2, w, par.T, pi1, pi2, par.alpha, par.nu, par.epsilon)
    
    # Market clearing
    labor_clearing = l1 + l2 - l
    good1_clearing = y1 - c1
    good2_clearing = y2 - c2
    
    return labor_clearing, good1_clearing, good2_clearing

# Prices
p1_values = np.linspace(0.1, 2.0, 10)
p2_values = np.linspace(0.1, 2.0, 10)

# Check market clearing conditions for each combination of p1 and p2
results = []
for p1 in p1_values:
    for p2 in p2_values:
        labor_clearing, good1_clearing, good2_clearing = market_clearing(p1, p2, 1, par)
        results.append({
            'p1': p1,
            'p2': p2,
            'labor_clearing': labor_clearing,
            'good1_clearing': good1_clearing,
            'good2_clearing': good2_clearing
        })

# Create DataFrame for results
df_results = pd.DataFrame(results)

# Display DataFrame
from IPython.display import display

display(df_results)


,p1,p2,labor_clearing,good1_clearing,good2_clearing
0,0.1,0.100000,-1.906099,-3.848683,-9.113594
1,0.1,0.311111,-1.415066,-4.211002,-2.922140
2,0.1,0.522222,-1.091209,-4.458316,-1.514472
3,0.1,0.733333,-0.739855,-4.658865,-0.780851
4,0.1,0.944444,-0.326397,-4.832080,-0.274069
...,...,...,...,...,...
95,2.0,1.155556,4.371791,1.663311,-0.204148
96,2.0,1.366667,4.934827,1.656256,0.192905
97,2.0,1.577778,5.581507,1.649727,0.541758
98,2.0,1.788889,6.313371,1.643622,0.859207


Blah Blah Blah

**Question 2:** Find the equilibrium prices $p_1$ and $p_2$.<br>
*Hint: you can use Walras' law to only check 2 of the market clearings*

In [24]:
import numpy as np
from scipy.optimize import fsolve
from types import SimpleNamespace

# Define parameters
par = SimpleNamespace()
par.A = 1.0
par.gamma = 0.5
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0
par.tau = 0.0
par.T = 0.0

# Define functions
def optimal_labor(w, p, A, gamma):
    return (p * A**gamma / w)**(1 / (1 - gamma))

def optimal_output(A, l, gamma):
    return A * l**gamma

def firm_profit(w, p, A, gamma):
    return w * (p * A**gamma / w)**(1 - gamma) * (1 - gamma)

def consumer_utility(p1, p2, w, T, pi1, pi2, alpha, nu, epsilon):
    c1 = alpha * (w + T + pi1 + pi2) / p1
    c2 = (1 - alpha) * (w + T + pi1 + pi2) / (p2 + par.tau)
    l = ((c1**alpha * c2**(1 - alpha))**(1 / nu))**(1 / (1 + epsilon))
    return c1, c2, l

# Market clearing conditions
def market_clearing_conditions(prices, w, par):
    p1, p2 = prices
    
    # Firm 1
    l1 = optimal_labor(w, p1, par.A, par.gamma)
    y1 = optimal_output(par.A, l1, par.gamma)
    pi1 = firm_profit(w, p1, par.A, par.gamma)
    
    # Firm 2
    l2 = optimal_labor(w, p2, par.A, par.gamma)
    y2 = optimal_output(par.A, l2, par.gamma)
    pi2 = firm_profit(w, p2, par.A, par.gamma)
    
    # Consumer
    c1, c2, l = consumer_utility(p1, p2, w, par.T, pi1, pi2, par.alpha, par.nu, par.epsilon)
    
    # Market clearing
    labor_clearing = l1 + l2 - l
    good1_clearing = y1 - c1
    
    return [labor_clearing, good1_clearing]  # Check only two conditions

# Initial guess for p1 and p2
initial_guess = [1.0, 1.0]

# Solve for market clearing prices
solution = fsolve(market_clearing_conditions, initial_guess, args=(1, par))

# Display the solution
print("Market clearing prices:")
print(f"p1 = {solution[0]}")
print(f"p2 = {solution[1]}")

# Check the market clearing conditions at the solution
labor_clearing, good1_clearing = market_clearing_conditions(solution, 1, par)
good2_clearing = optimal_output(par.A, optimal_labor(1, solution[1], par.A, par.gamma), par.gamma) - consumer_utility(solution[0], solution[1], 1, par.T, firm_profit(1, solution[0], par.A, par.gamma), firm_profit(1, solution[1], par.A, par.gamma), par.alpha, par.nu, par.epsilon)[1]

print("\nMarket clearing conditions at the solution:")
print(f"Labor clearing: {labor_clearing}")
print(f"Good 1 clearing: {good1_clearing}")
print(f"Good 2 clearing: {good2_clearing}")


Market clearing prices:
p1 = 0.7475486222511643
p2 = 0.7411786704525158

Market clearing conditions at the solution:
Labor clearing: 5.557554416668609e-12
Good 1 clearing: -2.6187940704858192e-12
Good 2 clearing: -1.0180924087834382


Blah blah - vi tester.

In [25]:
import numpy as np

# Given equilibrium prices
p1 = 0.7475486222511643
p2 = 0.7411786704525158

# Parameters
alpha = 0.5
gamma = 0.5
A = 1
w = 1  # numeraire

# Function to calculate optimal labor allocation
def calculate_l_star(p1, p2, A, gamma, w):
    ell1_star = (p1 * A * gamma / w) ** (1 / (1 - gamma))
    ell2_star = (p2 * A * gamma / w) ** (1 / (1 - gamma))
    return ell1_star, ell2_star

# Function to calculate optimal outputs
def calculate_y_star(ell1_star, ell2_star, A, gamma):
    y1_star = A * (ell1_star ** gamma)
    y2_star = A * (ell2_star ** gamma)
    return y1_star, y2_star

# Function to calculate profits
def calculate_profits(p1, p2, A, gamma, w):
    pi1_star = (1 - gamma) / gamma * w * (p1 * A * gamma / w) ** (1 / (1 - gamma))
    pi2_star = (1 - gamma) / gamma * w * (p2 * A * gamma / w) ** (1 / (1 - gamma))
    return pi1_star, pi2_star

# Calculate optimal labor allocation
ell1_star, ell2_star = calculate_l_star(p1, p2, A, gamma, w)

# Calculate optimal outputs
y1_star, y2_star = calculate_y_star(ell1_star, ell2_star, A, gamma)

# Calculate implied profits
pi1_star, pi2_star = calculate_profits(p1, p2, A, gamma, w)

# Total labor supplied
ell_star_val = ell1_star + ell2_star

# Calculate optimal consumption given T = 0
T = 0
c1_star = alpha * (w * ell_star_val + T + pi1_star + pi2_star) / p1
c2_star = (1 - alpha) * (w * ell_star_val + T + pi1_star + pi2_star) / p2

# Print results to check market clearing
print(f"Optimal labor (ell1*, ell2*): ({ell1_star}, {ell2_star})")
print(f"Optimal output (y1*, y2*): ({y1_star}, {y2_star})")
print(f"Implied profits (pi1*, pi2*): ({pi1_star}, {pi2_star})")
print(f"Total labor supplied (ell*): {ell_star_val}")
print(f"Optimal consumption (c1*, c2*): ({c1_star}, {c2_star})")

# Tolerance for approximate equality
tolerance = 1e-2

# Check market clearing conditions
labor_market_clears = np.abs(ell_star_val - (ell1_star + ell2_star)) < tolerance
goods_market_1_clears = np.abs(c1_star - y1_star) < tolerance
goods_market_2_clears = np.abs(c2_star - y2_star) < tolerance

print(f"Labor market clears: {labor_market_clears}")
print(f"Goods market 1 clears: {goods_market_1_clears}")
print(f"Goods market 2 clears: {goods_market_2_clears}")


Optimal labor (ell1*, ell2*): (0.13970723565740348, 0.13733645538343978)
Optimal output (y1*, y2*): (0.3737743111255821, 0.3705893352262579)
Implied profits (pi1*, pi2*): (0.13970723565740348, 0.13733645538343978)
Total labor supplied (ell*): 0.27704369104084325
Optimal consumption (c1*, c2*): (0.3706029050077776, 0.3737879975306066)
Labor market clears: True
Goods market 1 clears: True
Goods market 2 clears: True


Assume the government care about the social welfare function:

$$
SWF = U - \kappa y_2^*
$$

Here $\kappa$ measures the social cost of carbon emitted by the production of $y_2$ in equilibrium.

**Question 3:** What values of $\tau$ and (implied) $T$ should the government choose to maximize $SWF$?

In [26]:
import numpy as np
from scipy.optimize import minimize

# Parameters
alpha = 0.5  # assuming an example value
nu = 1.0  # assuming an example value
epsilon = 1.0  # assuming an example value
kappa = 0.1  # assuming an example value
gamma = 0.5  # assuming an example value
A = 1.0  # assuming an example value
p1 = 1.0  # assuming an example value
p2 = 1.0  # assuming an example value
w = 1.0  # assuming w is the numeraire
tau_initial_guess = 0.0

# Function to calculate optimal labor
def ell_star(p1, p2, tau):
    ell1 = (p1 * A * gamma / w) ** (1 / (1 - gamma))
    ell2 = (p2 * A * gamma / w) ** (1 / (1 - gamma))
    return ell1 + ell2

# Function to calculate T given optimal tau
def calculate_T(optimal_tau):
    ell_star_val = ell_star(p1, p2, optimal_tau)
    pi1_star = (1 - gamma) / gamma * w * (p1 * A * gamma / w) ** (1 / (1 - gamma))
    pi2_star = (1 - gamma) / gamma * w * (p2 * A * gamma / w) ** (1 / (1 - gamma))

    T = optimal_tau * (1 - alpha) * (w * ell_star_val + pi1_star + pi2_star) / (p2 + optimal_tau)
    return T

# Define the objective function (negative SWF to maximize)
def objective(tau):
    ell_star_val = ell_star(p1, p2, tau)
    pi1_star = (1 - gamma) / gamma * w * (p1 * A * gamma / w) ** (1 / (1 - gamma))
    pi2_star = (1 - gamma) / gamma * w * (p2 * A * gamma / w) ** (1 / (1 - gamma))

    T_val = tau * (1 - alpha) * (w * ell_star_val + pi1_star + pi2_star) / (p2 + tau)
    c1_star = alpha * (w * ell_star_val + T_val + pi1_star + pi2_star) / p1
    c2_star = (1 - alpha) * (w * ell_star_val + T_val + pi1_star + pi2_star) / (p2 + tau)

    SWF = np.log(c1_star ** alpha * c2_star ** (1 - alpha)) - nu * (ell_star_val ** (1 + epsilon)) / (1 + epsilon) - kappa * c2_star
    return -SWF  # negative because we are maximizing

# Optimize tau
result = minimize(objective, tau_initial_guess, method='Nelder-Mead')
optimal_tau = result.x[0]

# Calculate T using the optimal tau
optimal_T = calculate_T(optimal_tau)

print(f"Optimal tau: {optimal_tau}")
print(f"Implied optimal T: {optimal_T}")


Optimal tau: 0.03631250000000001
Implied optimal T: 0.017520053072794167


In [27]:
# Parameters
alpha = 0.5
nu = 1.0
epsilon = 1.0
kappa = 0.1
gamma = 0.5
A = 1.0
p1 = 1.0
p2 = 1.0
w = 1.0
tau_initial_guess = 0.0

# Function to calculate optimal labor
def ell_star(p1, p2, tau):
    ell1 = (p1 * A * gamma / w) ** (1 / (1 - gamma))
    ell2 = (p2 * A * gamma / w) ** (1 / (1 - gamma))
    return ell1 + ell2

# Function to calculate T given optimal tau
def calculate_T(optimal_tau):
    ell_star_val = ell_star(p1, p2, optimal_tau)
    pi1_star = (1 - gamma) / gamma * w * (p1 * A * gamma / w) ** (1 / (1 - gamma))
    pi2_star = (1 - gamma) / gamma * w * (p2 * A * gamma / w) ** (1 / (1 - gamma))

    T = optimal_tau * (1 - alpha) * (w * ell_star_val + pi1_star + pi2_star) / (p2 + optimal_tau)
    return T

# Define the objective function (negative SWF to maximize)
def objective(tau):
    ell_star_val = ell_star(p1, p2, tau)
    pi1_star = (1 - gamma) / gamma * w * (p1 * A * gamma / w) ** (1 / (1 - gamma))
    pi2_star = (1 - gamma) / gamma * w * (p2 * A * gamma / w) ** (1 / (1 - gamma))

    T_val = tau * (1 - alpha) * (w * ell_star_val + pi1_star + pi2_star) / (p2 + tau)
    c1_star = alpha * (w * ell_star_val + T_val + pi1_star + pi2_star) / p1
    c2_star = (1 - alpha) * (w * ell_star_val + T_val + pi1_star + pi2_star) / (p2 + tau)

    SWF = np.log(c1_star ** alpha * c2_star ** (1 - alpha)) - nu * (ell_star_val ** (1 + epsilon)) / (1 + epsilon) - kappa * c2_star
    return -SWF  # negative because we are maximizing

# Optimize tau
result = minimize(objective, tau_initial_guess, method='Nelder-Mead')
optimal_tau = result.x[0]

# Calculate T using the optimal tau
optimal_T = calculate_T(optimal_tau)

# Verify the results make sense
ell_star_val = ell_star(p1, p2, optimal_tau)
pi1_star = (1 - gamma) / gamma * w * (p1 * A * gamma / w) ** (1 / (1 - gamma))
pi2_star = (1 - gamma) / gamma * w * (p2 * A * gamma / w) ** (1 / (1 - gamma))

# Compute optimal consumption
T_val = optimal_tau * (1 - alpha) * (w * ell_star_val + pi1_star + pi2_star) / (p2 + optimal_tau)
c1_star = alpha * (w * ell_star_val + T_val + pi1_star + pi2_star) / p1
c2_star = (1 - alpha) * (w * ell_star_val + T_val + pi1_star + pi2_star) / (p2 + optimal_tau)

# Compute SWF
SWF = np.log(c1_star ** alpha * c2_star ** (1 - alpha)) - nu * (ell_star_val ** (1 + epsilon)) / (1 + epsilon) - kappa * c2_star

# Print results
print(f"Optimal tau: {optimal_tau}")
print(f"Implied optimal T: {optimal_T}")
print(f"Optimal labor (ell*): {ell_star_val}")
print(f"Optimal consumption c1*: {c1_star}")
print(f"Optimal consumption c2*: {c2_star}")
print(f"Social Welfare Function (SWF): {SWF}")


Optimal tau: 0.03631250000000001
Implied optimal T: 0.017520053072794167
Optimal labor (ell*): 0.5
Optimal consumption c1*: 0.5087600265363971
Optimal consumption c2*: 0.4909330212039294
Social Welfare Function (SWF): -0.8677065060000058


In [28]:
import numpy as np
from scipy.optimize import minimize

# Parameters
alpha = 0.5
nu = 1.0
epsilon = 1.0
kappa = 0.1
gamma = 0.5
A = 1.0
p1 = 1.0
p2 = 1.0
w = 1.0
tau_initial_guess = 0.0

# Function to calculate optimal labor
def ell_star(p1, p2, tau):
    ell1 = (p1 * A * gamma / w) ** (1 / (1 - gamma))
    ell2 = (p2 * A * gamma / w) ** (1 / (1 - gamma))
    return ell1 + ell2

# Function to calculate T given optimal tau
def calculate_T(optimal_tau):
    ell_star_val = ell_star(p1, p2, optimal_tau)
    pi1_star = (1 - gamma) / gamma * w * (p1 * A * gamma / w) ** (1 / (1 - gamma))
    pi2_star = (1 - gamma) / gamma * w * (p2 * A * gamma / w) ** (1 / (1 - gamma))

    T = optimal_tau * (1 - alpha) * (w * ell_star_val + pi1_star + pi2_star) / (p2 + optimal_tau)
    return T

# Define the objective function (negative SWF to maximize)
def objective(tau):
    ell_star_val = ell_star(p1, p2, tau)
    pi1_star = (1 - gamma) / gamma * w * (p1 * A * gamma / w) ** (1 / (1 - gamma))
    pi2_star = (1 - gamma) / gamma * w * (p2 * A * gamma / w) ** (1 / (1 - gamma))

    T_val = tau * (1 - alpha) * (w * ell_star_val + pi1_star + pi2_star) / (p2 + tau)
    c1_star = alpha * (w * ell_star_val + T_val + pi1_star + pi2_star) / p1
    c2_star = (1 - alpha) * (w * ell_star_val + T_val + pi1_star + pi2_star) / (p2 + tau)

    SWF = np.log(c1_star ** alpha * c2_star ** (1 - alpha)) - nu * (ell_star_val ** (1 + epsilon)) / (1 + epsilon) - kappa * c2_star
    return -SWF  # negative because we are maximizing

# Optimize tau
result = minimize(objective, tau_initial_guess, method='Nelder-Mead')
optimal_tau = result.x[0]

# Calculate T using the optimal tau
optimal_T = calculate_T(optimal_tau)

# Verify the results make sense
ell_star_val = ell_star(p1, p2, optimal_tau)
pi1_star = (1 - gamma) / gamma * w * (p1 * A * gamma / w) ** (1 / (1 - gamma))
pi2_star = (1 - gamma) / gamma * w * (p2 * A * gamma / w) ** (1 / (1 - gamma))

# Compute optimal consumption
T_val = optimal_tau * (1 - alpha) * (w * ell_star_val + pi1_star + pi2_star) / (p2 + optimal_tau)
c1_star = alpha * (w * ell_star_val + T_val + pi1_star + pi2_star) / p1
c2_star = (1 - alpha) * (w * ell_star_val + T_val + pi1_star + pi2_star) / (p2 + optimal_tau)

# Compute SWF
SWF = np.log(c1_star ** alpha * c2_star ** (1 - alpha)) - nu * (ell_star_val ** (1 + epsilon)) / (1 + epsilon) - kappa * c2_star

# Print results
print(f"Optimal tau: {optimal_tau}")
print(f"Implied optimal T: {optimal_T}")
print(f"Optimal labor (ell*): {ell_star_val}")
print(f"Optimal consumption c1*: {c1_star}")
print(f"Optimal consumption c2*: {c2_star}")
print(f"Social Welfare Function (SWF): {SWF}")

# Components of SWF
U_c = np.log(c1_star ** alpha * c2_star ** (1 - alpha))
D_ell = -nu * (ell_star_val ** (1 + epsilon)) / (1 + epsilon)
C_y = -kappa * c2_star

# Calculate each part
print(f"Utility from consumption (U_c): {U_c}")
print(f"Disutility from labor (D_ell): {D_ell}")
print(f"Social cost of carbon (C_y): {C_y}")

# Verify the SWF calculation
calculated_SWF = U_c + D_ell + C_y
print(f"Calculated SWF: {calculated_SWF}")


Optimal tau: 0.03631250000000001
Implied optimal T: 0.017520053072794167
Optimal labor (ell*): 0.5
Optimal consumption c1*: 0.5087600265363971
Optimal consumption c2*: 0.4909330212039294
Social Welfare Function (SWF): -0.8677065060000058
Utility from consumption (U_c): -0.6936132038796128
Disutility from labor (D_ell): -0.125
Social cost of carbon (C_y): -0.049093302120392944
Calculated SWF: -0.8677065060000058


## 2. <a id='toc2_'></a>[Problem 2: Career choice model](#toc0_)

Consider a graduate $i$ making a choice between entering $J$ different career tracks. <br>
Entering career $j$ yields utility $u^k_{ij}$. This value is unknown to the graduate ex ante, but will ex post be: <br>
$$
    u_{i,j}^k = v_{j} + \epsilon_{i,j}^k
$$

They know that $\epsilon^k_{i,j}\sim \mathcal{N}(0,\sigma^2)$, but they do not observe $\epsilon^k_{i,j}$ before making their career choice. <br>

Consider the concrete case of $J=3$ with:
$$
\begin{align*}
    v_{1} &= 1 \\
    v_{2} &= 2 \\
    v_{3} &= 3
\end{align*}
$$

If the graduates know the values of $v_j$ and the distribution of $\epsilon_{i,j}^k$, they can calculate the expected utility of each career track using simulation: <br>
$$
    \mathbb{E}\left[ u^k_{i,j}\vert v_j \right] \approx v_j + \frac{1}{K}\sum_{k=1}^K \epsilon_{i,j}^k
$$

In [29]:
par = SimpleNamespace()
par.J = 3
par.N = 10
par.K = 10000

par.F = np.arange(1,par.N+1)
par.sigma = 2

par.v = np.array([1,2,3])
par.c = 1

**Question 1:** Simulate and calculate expected utility and the average realised utility for $K=10000$ draws, for each career choice $j$.


In [30]:
# write your answer here

Now consider a new scenario: Imagine that the graduate does not know $v_j$. The *only* prior information they have on the value of each job, comes from their $F_{i}$ friends that work in each career $j$. After talking with them, they know the average utility of their friends (which includes their friends' noise term), giving them the prior expecation: <br>
$$
\tilde{u}^k_{i,j}\left( F_{i}\right) = \frac{1}{F_{i}}\sum_{f=1}^{F_{i}} \left(v_{j} + \epsilon^k_{f,j}\right), \; \epsilon^k_{f,j}\sim \mathcal{N}(0,\sigma^2)
$$
For ease of notation consider that each graduate have $F_{i}=i$ friends in each career. <br>

For $K$ times do the following: <br>
1. For each person $i$ draw $J\cdot F_i$ values of $\epsilon_{f,j}^{k}$, and calculate the prior expected utility of each career track, $\tilde{u}^k_{i,j}\left( F_{i}\right)$. <br>
Also draw their own $J$ noise terms, $\epsilon_{i,j}^k$
1. Each person $i$ chooses the career track with the highest expected utility: $$j_i^{k*}= \arg\max_{j\in{1,2\dots,J}}\left\{ \tilde{u}^k_{i,j}\left( F_{i}\right)\right\} $$
1. Store the chosen careers: $j_i^{k*}$, the prior expectation of the value of their chosen career: $\tilde{u}^k_{i,j=j_i^{k*}}\left( F_{i}\right)$, and the realized value of their chosen career track: $u^k_{i,j=j_i^{k*}}=v_{j=j_i^{k*}}+\epsilon_{i,j=j_i^{k*}}^k$.

Chosen values will be: <br>
$i\in\left\{1,2\dots,N\right\}, N=10$ <br>
$F_i = i$<br>
So there are 10 graduates. The first has 1 friend in each career, the second has 2 friends, ... the tenth has 10 friends.

**Question 2:** Simulate and visualize: For each type of graduate, $i$, the share of graduates choosing each career, the average subjective expected utility of the graduates, and the average ex post realized utility given their choice. <br>
That is, calculate and visualize: <br>
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} \mathbb{I}\left\{ j=j_i^{k*} \right\}  \;\forall j\in\left\{1,2,\dots,J\right\}
\end{align*}
$$
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} \tilde{u}^k_{ij=j_i^{k*}}\left( F_{i}\right)
\end{align*}
$$
And 
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} u^k_{ij=j_i^{k*}} 
\end{align*}
$$
For each graduate $i$.

In [31]:
# Write your answer here 

After a year of working in their career, the graduates learn $u^k_{ij}$ for their chosen job $j_i^{k*}$ perfectly. <br>
The can switch to one of the two remaining careers, for which they have the same prior as before, but it will now include a switching cost of $c$ which is known.
Their new priors can be written as: 
$$
\tilde{u}^{k,2}_{ij}\left( F_{i}\right) = \begin{cases}
            \tilde{u}^k_{ij}\left( F_{i}\right)-c & \text{if } j \neq j_i^{k*} \\
            u_{ij=j_i^{k*}} & \text{if } j = j_i^{k*}
        \end{cases}
$$

We will set $c=1$.

Their realized utility will be: <br>
$$
u^{k,2}_{ij}= \begin{cases}
            u_{ij}^k -c & \text{if } j \neq j_i^{k*} \\
            u_{ij=j_i^{k*}} & \text{if } j = j_i^{k*}
        \end{cases}
$$

**Question 3:** Following the same approach as in question 2, find the new optimal career choice for each $i$, $k$. Then for each $i$, calculate the average subjective expected utility from their new optimal career choice, and the ex post realized utility of that career. Also, for each $i$, calculate the share of graduates that chooses to switch careers, conditional on which career they chose in the first year. <br>

In [32]:
# write your answer here

## 3. <a id='toc3_'></a>[Problem 3: Barycentric interpolation](#toc0_)

**Problem:** We have a set of random points in the unit square,

$$
\mathcal{X} = \{(x_1,x_2)\,|\,x_1\sim\mathcal{U}(0,1),x_2\sim\mathcal{U}(0,1)\}.
$$

For these points, we know the value of some function $f(x_1,x_2)$,

$$
\mathcal{F} = \{f(x_1,x_2) \,|\, (x_1,x_2) \in \mathcal{X}\}.
$$

Now we want to approximate the value $f(y_1,y_2)$ for some  $y=(y_1,y_2)$, where $y_1\sim\mathcal{U}(0,1)$ and $y_2\sim\mathcal{U}(0,1)$.

**Building block I**

For an arbitrary triangle $ABC$ and a point $y$, define the so-called barycentric coordinates as:

$$
\begin{align*}
  r^{ABC}_1 &= \frac{(B_2-C_2)(y_1-C_1) + (C_1-B_1)(y_2-C_2)}{(B_2-C_2)(A_1-C_1) + (C_1-B_1)(A_2-C_2)} \\
  r^{ABC}_2 &= \frac{(C_2-A_2)(y_1-C_1) + (A_1-C_1)(y_2-C_2)}{(B_2-C_2)(A_1-C_1) + (C_1-B_1)(A_2-C_2)} \\
  r^{ABC}_3 &= 1 - r_1 - r_2.
\end{align*}
$$

If $r^{ABC}_1 \in [0,1]$, $r^{ABC}_2 \in [0,1]$, and $r^{ABC}_3 \in [0,1]$, then the point is inside the triangle.

We always have $y = r^{ABC}_1 A + r^{ABC}_2 B + r^{ABC}_3 C$.

**Building block II**

Define the following points:

$$
\begin{align*}
A&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}>y_{1}\text{ and }x_{2}>y_{2}\\
B&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}>y_{1}\text{ and }x_{2}<y_{2}\\
C&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}<y_{1}\text{ and }x_{2}<y_{2}\\
D&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}<y_{1}\text{ and }x_{2}>y_{2}.
\end{align*}
$$

**Algorithm:**

1. Compute $A$, $B$, $C$, and $D$. If not possible return `NaN`.
1. If $y$ is inside the triangle $ABC$ return $r^{ABC}_1 f(A) + r^{ABC}_2 f(B) + r^{ABC}_3 f(C)$.
1. If $y$ is inside the triangle $CDA$ return $r^{CDA}_1 f(C) + r^{CDA}_2 f(D) + r^{CDA}_3 f(A)$.
1. Return `NaN`.



**Sample:**

In [33]:
rng = np.random.default_rng(2024)

X = rng.uniform(size=(50,2))
y = rng.uniform(size=(2,))


**Questions 1:** Find $A$, $B$, $C$ and $D$. Illustrate these together with $X$, $y$ and the triangles $ABC$ and $CDA$.

In [34]:
# write your answer here

**Question 2:** Compute the barycentric coordinates of the point $y$ with respect to the triangles $ABC$ and $CDA$. Which triangle is $y$ located inside?

In [35]:
# write your answer here

Now consider the function:
$$
f(x_1,x_2) = x_1 \cdot x_2
$$

In [36]:
f = lambda x: x[0]*x[1]
F = np.array([f(x) for x in X])

**Question 3:** Compute the approximation of $f(y)$ using the full algorithm. Compare with the true value.

In [37]:
# write your answer here

**Question 4:** Repeat question 3 for all points in the set $Y$.

In [38]:
Y = [(0.2,0.2),(0.8,0.2),(0.8,0.8),(0.8,0.2),(0.5,0.5)]

In [39]:
# write your answer here